# Importing libraries and modules (Data Collection)


In [5]:
# Importing the libraries and modules necessary for our classification model.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# Storing the dataset into a data frame variable.

data_frame = pd.read_csv('Dataset_7.csv')
data_frame

,Unnamed: 0,Symptom,Disease
0,0,"anxiety and nervousness ,shortness of breath ,...",panic disorder
1,1,"shortness of breath ,depressive or psychotic s...",panic disorder
2,2,"anxiety and nervousness ,depression ,shortness...",panic disorder
3,3,"anxiety and nervousness ,depressive or psychot...",panic disorder
4,4,"anxiety and nervousness ,depression ,insomnia ...",panic disorder
...,...,...,...
101898,101898,"itching of skin ,skin dryness, peeling, scalin...",eczema
101899,101899,skin rash,eczema
101900,101900,"skin lesion ,allergic reaction ,itching of ski...",eczema
101901,101901,"cough ,abnormal appearing skin ,skin dryness, ...",eczema


# Data Preprocessing


In [7]:
# 1. Taking care of missing data in the dataset.

# Attaining the sum of empty/missing values.
missing_values = data_frame.isnull().sum().sum()

if missing_values == 0:
  print("There is no missing data in the dataset.")
else:
  data_frame.dropna(inplace=True) # Dropping the missing entries.

There is no missing data in the dataset.


In [8]:
import re

# Function to clean symptom names dynamically (regular expressions)
def clean_symptoms(symptoms):
    # Removing trailing numeric suffixes and unwanted symbols
    return [re.sub(r'\d+$', '', re.sub(r'[^\w\s]', '', symptom.strip())) for symptom in symptoms]

In [9]:
from sklearn.preprocessing import MultiLabelBinarizer  # Reference: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html

"""
# 2. Applying MultiLabelBinarizer (version of multi-labeled one-hot encoding)
   - Used for encoding multi-labeled categorical data (e.g., multiple symptoms).
"""
# Initialize MultiLabelBinarizer for multi-labeled one-hot encoding
mlb = MultiLabelBinarizer()

# Splitting the symptoms column into lists for encoding
# Clean_symptoms is assumed to handle any preprocessing such as trimming or standardization
query_symptom_column = data_frame['Symptom'].str.split(',').apply(lambda x: clean_symptoms(x))

# Using MultiLabelBinarizer for encoding
query_encoded = mlb.fit_transform(query_symptom_column)

# Extracting the encoded column names from MultiLabelBinarizer
encoded_columns = mlb.classes_

# Converting the encoded data back to a DataFrame for better visualization
query_encoded_df = pd.DataFrame(query_encoded, columns=encoded_columns)

# Concatenating the one-hot encoded symptoms back to the original dataset
# Replacing the original 'Symptom' column with the encoded features
data_frame = pd.concat([data_frame.drop(columns=['Symptom']), query_encoded_df], axis=1)

In [10]:
# Ensuring the 'Disease' column is the last column (as it is the label)
columns = [col for col in data_frame.columns if col != 'Disease'] + ['Disease']
data_frame = data_frame[columns]
data_frame.drop(columns=['Unnamed: 0'], inplace=True)
data_frame

<ipython-input-10-65f61ae389f1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame.drop(columns=['Unnamed: 0'], inplace=True)


,abnormal appearing skin,abnormal breathing sounds,abnormal involuntary movements,abnormal movement of eyelid,abusing alcohol,ache all over,acne or pimples,allergic reaction,ankle pain,ankle swelling,...,vomiting,vomiting blood,warts,weakness,weight gain,wheezing,white discharge from eye,wrist pain,wrist swelling,Disease
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,panic disorder
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,panic disorder
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,panic disorder
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,panic disorder
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,panic disorder
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101898,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,eczema
101899,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,eczema
101900,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,eczema
101901,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,eczema


In [11]:
# Converting the data frame into a numpy array.

numpy_array = data_frame.to_numpy()
numpy_array

array([[0, 0, 0, ..., 0, 0, 'panic disorder'],
       [0, 0, 0, ..., 0, 0, 'panic disorder'],
       [0, 0, 0, ..., 0, 0, 'panic disorder'],
       ...,
       [0, 0, 0, ..., 0, 0, 'eczema'],
       [1, 0, 0, ..., 0, 0, 'eczema'],
       [0, 0, 0, ..., 0, 0, 'eczema']], dtype=object)

In [12]:
# Splitting the dataset into training set and testing set (70/30 split)
from sklearn.model_selection import train_test_split  # Reference: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

X = data_frame.drop(columns=['Disease'])  # Features
y = data_frame['Disease']  # Label

# 70/30 split, where stratify ensures the class distribution remains consistent across splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Confirming the class distribution in training and testing sets
# Useful for verifying that the stratify parameter preserved class proportions
train_distribution = y_train.value_counts()
test_distribution = y_test.value_counts()

print("Training Set Distribution:\n", train_distribution)
print("\nTesting Set Distribution:\n", test_distribution)

Training Set Distribution:
 Disease
cystitis                          853
nose disorder                     853
vulvodynia                        853
complex regional pain syndrome    852
hypoglycemia                      851
                                 ... 
rectal disorder                   631
hypertensive heart disease        631
psychotic disorder                631
pelvic inflammatory disease       630
chronic back pain                 629
Name: count, Length: 100, dtype: int64

Testing Set Distribution:
 Disease
cystitis                          366
complex regional pain syndrome    365
nose disorder                     365
vulvodynia                        365
spondylosis                       365
                                 ... 
asthma                            271
psychotic disorder                271
pyelonephritis                    271
pelvic inflammatory disease       270
chronic back pain                 270
Name: count, Length: 100, dtype: int64


# Model Selection

In [13]:
# Importing metrics for prediction results
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
# References:
# - Classification Report: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
# - Confusion Matrix: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
# - Accuracy Score: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
# - F1 Score: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

## First Model (Decision Trees)

In [14]:
from sklearn.tree import DecisionTreeClassifier  # Reference: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
# Reference for metrics: https://scikit-learn.org/stable/modules/model_evaluation.html

# Initialising the Decision Tree Classifier
dt_model = DecisionTreeClassifier(random_state=42)  # Decision Tree with default parameters

# Training the model
dt_model.fit(X_train, y_train)

# Making predictions
dt_predictions = dt_model.predict(X_test)

# Evaluate on training data
train_predictions = dt_model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_predictions)
print("Training Accuracy:", train_accuracy)

# Evaluate on testing data
test_predictions = dt_model.predict(X_test)
test_accuracy = accuracy_score(y_test, test_predictions)
print("Testing Accuracy:", test_accuracy)

# Compare results
# Overfitting check: A large difference between training and testing accuracy indicates overfitting.
if train_accuracy - test_accuracy > 0.1:  # Example threshold
    print("Potentially Overfitted")
else:
    print("Model appears to generalize well.")

# Actual and predicted classes for F1-score calculation
y_true = ["panic disorder", "acute pancreatitis", "asthma", "heart attack"]
y_pred = ["panic disorder", "acute pancreatitis", "heart attack", "asthma"]

# Evaluating the model
print("Accuracy:", accuracy_score(y_test, dt_predictions))
print("\nClassification Report:\n", classification_report(y_test, dt_predictions))  # Scikit-learn documentation
print("\nConfusion Matrix:\n", confusion_matrix(y_test, dt_predictions))  # Scikit-learn documentation
f1_macro = f1_score(y_true, y_pred, average='macro')  # Scikit-learn documentation
print(f"F1-Score (Macro): {f1_macro:.4f}")

Training Accuracy: 0.96371894801772
Testing Accuracy: 0.8593111118380165
Potentially Overfitted
Accuracy: 0.8593111118380165

Classification Report:
                                               precision    recall  f1-score   support

                           actinic keratosis       0.72      0.77      0.74       273
                         acute bronchiolitis       0.87      0.93      0.90       361
                            acute bronchitis       0.65      0.79      0.72       364
                          acute bronchospasm       0.59      0.66      0.62       272
                         acute kidney injury       0.97      0.96      0.97       272
                          acute pancreatitis       0.83      0.89      0.85       362
                             acute sinusitis       0.80      0.87      0.83       272
                       acute stress reaction       0.87      0.90      0.89       273
                                     allergy       0.92      0.95      0.94

# Second Model (Random Forest Classifier)

In [ ]:
from sklearn.ensemble import RandomForestClassifier  # Reference: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
# Reference for metrics: https://scikit-learn.org/stable/modules/model_evaluation.html

# Initialising the Random Forest Classifier
rf_model = RandomForestClassifier(random_state=42)  # Random Forest with default parameters

# Training the model
rf_model.fit(X_train, y_train)

# Making predictions on the test set
rf_predictions = rf_model.predict(X_test)

# Evaluating on training data
rf_train_predictions = rf_model.predict(X_train)
rf_train_accuracy = accuracy_score(y_train, rf_train_predictions)
print("Training Accuracy:", rf_train_accuracy)

# Evaluating on testing data
rf_test_accuracy = accuracy_score(y_test, rf_predictions)
print("Testing Accuracy:", rf_test_accuracy)

# Checking for potential overfitting
# Overfitting check: A large difference between training and testing accuracy indicates overfitting.
if rf_train_accuracy - rf_test_accuracy > 0.1:
    print("Potentially Overfitted")
else:
    print("Model appears to generalize well.")

# Actual and predicted classes for F1-score calculation
y_true = ["panic disorder", "acute pancreatitis", "asthma", "heart attack"]
y_pred = ["panic disorder", "acute pancreatitis", "heart attack", "asthma"]

# Evaluate overall model performance
print("\nAccuracy:", rf_test_accuracy)
print("\nClassification Report:\n", classification_report(y_test, rf_predictions))  # Scikit-learn documentation
print("\nConfusion Matrix:\n", confusion_matrix(y_test, rf_predictions))  # Scikit-learn documentation
f1_macro = f1_score(y_true, y_pred, average='macro')  # Scikit-learn documentation
print(f"F1-Score (Macro): {f1_macro:.4f}")

# Third Model (Support Vector Machines)

In [ ]:
from sklearn.svm import SVC  # Reference: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
# Reference for metrics: https://scikit-learn.org/stable/modules/model_evaluation.html

# Initializing the Support Vector Machine Classifier
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)  # Radial Basis Function kernel is used by default

# Training the model
svm_model.fit(X_train, y_train)

# Making predictions on the test set
svm_predictions = svm_model.predict(X_test)

# Evaluating on training data
svm_train_predictions = svm_model.predict(X_train)
svm_train_accuracy = accuracy_score(y_train, svm_train_predictions)
print("Training Accuracy:", svm_train_accuracy)

# Evaluating on testing data
svm_test_accuracy = accuracy_score(y_test, svm_predictions)
print("Testing Accuracy:", svm_test_accuracy)

# Checking for potential overfitting
# Overfitting check: A large difference between training and testing accuracy indicates overfitting.
if svm_train_accuracy - svm_test_accuracy > 0.1:
    print("Potentially Overfitted")
else:
    print("Model appears to generalize well.")

# Actual and predicted classes for F1-score calculation
y_true = ["panic disorder", "acute pancreatitis", "asthma", "heart attack"]
y_pred = ["panic disorder", "acute pancreatitis", "heart attack", "asthma"]

# Evaluate overall model performance
print("\nAccuracy:", svm_test_accuracy)
print("\nClassification Report:\n", classification_report(y_test, svm_predictions))  # Scikit-learn documentation
print("\nConfusion Matrix:\n", confusion_matrix(y_test, svm_predictions))  # Scikit-learn documentation
f1_macro = f1_score(y_true, y_pred, average='macro')  # Scikit-learn documentation
print(f"F1-Score (Macro): {f1_macro:.4f}")

# Fourth Model (K-Nearest Neighbours)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier  # Reference: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
# Reference for metrics: https://scikit-learn.org/stable/modules/model_evaluation.html

# Initializing the K-Nearest Neighbors Classifier
knn_model = KNeighborsClassifier(n_neighbors=5)  # Uses the default distance metric (Minkowski)

# Training the model
knn_model.fit(X_train, y_train)

# Making predictions on the test set
knn_predictions = knn_model.predict(X_test)

# Evaluating on training data
knn_train_predictions = knn_model.predict(X_train)
knn_train_accuracy = accuracy_score(y_train, knn_train_predictions)
print("Training Accuracy:", knn_train_accuracy)

# Evaluating on testing data
knn_test_accuracy = accuracy_score(y_test, knn_predictions)
print("Testing Accuracy:", knn_test_accuracy)

# Checking for potential overfitting
# Overfitting check: A large difference between training and testing accuracy indicates overfitting.
if knn_train_accuracy - knn_test_accuracy > 0.1:
    print("Potentially Overfitted")
else:
    print("Model appears to generalize well.")

# Actual and predicted classes for F1-score calculation
y_true = ["panic disorder", "acute pancreatitis", "asthma", "heart attack"]
y_pred = ["panic disorder", "acute pancreatitis", "heart attack", "asthma"]

# Evaluate overall model performance
print("\nAccuracy:", knn_test_accuracy)
print("\nClassification Report:\n", classification_report(y_test, knn_predictions))  # Scikit-learn documentation
print("\nConfusion Matrix:\n", confusion_matrix(y_test, knn_predictions))  # Scikit-learn documentation
f1_macro = f1_score(y_true, y_pred, average='macro')  # Scikit-learn documentation
print(f"F1-Score (Macro): {f1_macro:.4f}")

# Fifth Model (Naive Bayes Classifier)

In [ ]:
from sklearn.naive_bayes import GaussianNB  # Reference: https://scikit-learn.org/stable/modules/naive_bayes.html
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
# Reference for metrics: https://scikit-learn.org/stable/modules/model_evaluation.html

# Initializing the Naive Bayes Classifier
nb_model = GaussianNB()

# Training the model
nb_model.fit(X_train, y_train)

# Making predictions on the test set
nb_predictions = nb_model.predict(X_test)

# Evaluating on training data
nb_train_predictions = nb_model.predict(X_train)
nb_train_accuracy = accuracy_score(y_train, nb_train_predictions)
print("Training Accuracy:", nb_train_accuracy)

# Evaluating on testing data
nb_test_accuracy = accuracy_score(y_test, nb_predictions)
print("Testing Accuracy:", nb_test_accuracy)

# Checking for potential overfitting
if nb_train_accuracy - nb_test_accuracy > 0.1:
    print("Potentially Overfitted")
else:
    print("Model appears to generalize well.")

# Actual and predicted classes for F1-score calculation
y_true = ["panic disorder", "acute pancreatitis", "asthma", "heart attack"]
y_pred = ["panic disorder", "acute pancreatitis", "heart attack", "asthma"]

# Evaluate overall model performance
print("\nAccuracy:", nb_test_accuracy)
print("\nClassification Report:\n", classification_report(y_test, nb_predictions))  # Scikit-learn documentation
print("\nConfusion Matrix:\n", confusion_matrix(y_test, nb_predictions))  # Scikit-learn documentation
f1_macro = f1_score(y_true, y_pred, average='macro')  # Scikit-learn documentation
print(f"F1-Score (Macro): {f1_macro:.4f}")